# Visualizing Topic Model

This is a work-in-progress Notebook providing different types of visualization for a topic model. The first visualization (`pyLDAvis`) allows a researcher to interact with the *vocabulary* of the corpus and its distribution over topics. pyLDAvis is an out-of-the-box package, a wrapper around the R-library `LDAvis`.

The second visualization employs the `bokeh` library and allows a researcher to interact with (and inspect) *documents* and the distribution of topics over documents. 

The data used in this notebook are scraped from [ORACC](http://oracc.org). Most of the techniques used here, however, may be applied to any set of documents.

# Dependencies and Versions
pyLDAvis 2.0 is incompatible with Pandas 0.19 (check for pyLDAvis 2.1.0 or later). The MDS and tSNE computations necessary for plotting the distribution of documents in the topic model need scikit-learn 0.18. This notebook was written for Python 3.5.

In [1]:
import re
import os
import warnings
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import MDS, TSNE
from gensim import corpora, models, utils
import gensim
import pyLDAvis.gensim
from scipy.spatial.distance import pdist, squareform
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, CustomJS, Title
from bokeh.models.widgets import Slider
from bokeh.plotting import figure, output_file, output_notebook, save, show, reset_output
from bokeh.layouts import widgetbox, column
output_notebook()

Loading BokehJS ...

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [2]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [3]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.

In [4]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have less than 10 lemmas left and use that selection for the list `texts` and for the dataframe `df`. 

In [5]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [6]:
len(bo), len(df)

(4976, 3006)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
## CHECK - is this done correctly?

In [8]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

## TODO use slider 

In [9]:
ntopics = int(input("Number of topics: ") or 10)

Number of topics:  


In [10]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top words and their probabilities in all topics. Note: the topic numbers here are not the ones used below in the visualizations! (The topics are the same, but not their numbers).

In [11]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('qû[unit]n', 0.039634343),
   ('ilu[god]n', 0.02959446),
   ('šikaru[beer]n', 0.024415387),
   ('karānu[vine]n', 0.022837134),
   ('šamnu[oil]n', 0.020545267),
   ('kusāpu[bread]n', 0.017754514),
   ('zamāru[sing]v', 0.016015612),
   ('dišpu[honey]n', 0.014499569),
   ('nīqu[offering]n', 0.01421817),
   ('naqû[pour-(a-libation)]v', 0.014005929)]),
 (1,
  [('nišu[people]n', 0.06957513),
   ('imēru[unit]n', 0.06602787),
   ('eqlu[field]n', 0.057719853),
   ('immeru[sheep]n', 0.04232107),
   ('ikkaru[farmer]n', 0.041318834),
   ('alpu[ox]n', 0.03420751),
   ('kirû[garden]n', 0.03134715),
   ('sinništu[woman]n', 0.02788458),
   ('tarbaṣu[courtyard]n', 0.025301296),
   ('lawû[surround]v', 0.024567384)]),
 (2,
  [('rabû[big-one]n', 0.072569),
   ('pīhātu[responsibility]n', 0.039937504),
   ('ša-rēši[eunuch]n', 0.024973528),
   ('ša-qurbūti[close-follower]n', 0.024012562),
   ('ṣābu[people]n', 0.02039524),
   ('ēkallu[palace]n', 0.019527601),
   ('šaknu[appointee]n', 0.018716713),
  

# TODO
Move all visualization to separate notebook. Compute the model, show topics and show/explain document/topic and topic/term tables.

# pyLDAvis
Use pyLDAvis to visualize the topic model. By default, pyLDAvis will order the topics by [prevalence](https://github.com/bmabey/pyLDAvis/issues/59) (topic 1 is the most prevalent topic). That means that the topic numbers in the visualization do not agree with the topic numbers in the lda model. To prevent this behaviour one may use `sort_topics=False` in the `prepare` command. The advantage of ordering the topics by prevalence, however, is that new instances of the lda model are more comparable (that is, the same topic will receive the same number). Note that the library was written in Java for R, and so the numbering in the visualization begins with 1 (not with 0). The topic numbers in the Document/Topic and Topic/Term matrices below will be adjusted to be compatible with the pyLDAvis visualization.

PyLDAvis needs a large output box. The `%%html` lines below create such a box (for the code see [here](http://stackoverflow.com/questions/18770504/resize-ipython-notebook-output-window)). 

%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>


In [12]:
warnings.simplefilter(action='ignore', category=FutureWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
if not os.path.exists('vis'):
    os.makedirs('vis')
pyLDAvis.save_html(vis, 'vis/lda_terms.html')
pyLDAvis.display(vis)

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [13]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.04493431),
 (1, 0.0038660169),
 (2, 0.04645144),
 (3, 0.0038658662),
 (4, 0.0038665088),
 (5, 0.003866354),
 (6, 0.08779682),
 (7, 0.099573135),
 (8, 0.003866014),
 (9, 0.70191354)]

# Create Document/Topic Probability Table
A Document/Topiuc probability table is a table (DataFrame), where each row represents a document and each column a topuic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [14]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102054,0.009094,0.009095,0.009094,0.825184
P224382,0.044934,0.003866,0.046452,0.003866,0.003867,0.003866,0.087798,0.099573,0.003866,0.701912
P224383,0.003334,0.003334,0.320279,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653047
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156542,0.790116
P224388,0.006669,0.006671,0.098219,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848422


# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

In [15]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

,1,2,3,4,5,6,7,8,9,10
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102054,0.009094,0.009095,0.009094,0.825184
P224382,0.044934,0.003866,0.046452,0.003866,0.003867,0.003866,0.087798,0.099573,0.003866,0.701912
P224383,0.003334,0.003334,0.320279,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653047
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156542,0.790116
P224388,0.006669,0.006671,0.098219,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848422
...,...,...,...,...,...,...,...,...,...,...
X900011,0.002942,0.002943,0.002942,0.002942,0.002942,0.002942,0.245074,0.002942,0.373348,0.360984
X900012,0.002084,0.002084,0.477691,0.002084,0.002084,0.002084,0.002084,0.002084,0.053262,0.454460
X900013,0.001164,0.001163,0.001164,0.001163,0.001164,0.001164,0.001164,0.001163,0.014563,0.976129
X900014,0.002273,0.002273,0.979539,0.002273,0.002273,0.002274,0.002273,0.002273,0.002275,0.002274


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [16]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. The DataFrames are concatenated to a single DataFrame. 

In [17]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(0, ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True)
t_t_df_.head()

,0,1,2,3,4,5,6,7,8,9
aban-bāšti[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
aban-lamassi[(a-precious-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000404,0.000014,0.000029,0.000004
aban-râmi['love'-stone]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
abati[(meaning-unknown)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000029,0.000028
abašmû[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004


Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [18]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

,aban-bāšti[(a-stone)]n,aban-lamassi[(a-precious-stone)]n,aban-râmi['love'-stone]n,abati[(meaning-unknown)]n,abašmû[(a-stone)]n,abbušu[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,ablu[brought]aj,...,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭīpu[addition]n,ṭīru[impression]n,ṭūbtu[peace]n,ṭūbu[goodness]n,ṭūbātu[happiness]n,ṭūdu[way]n,ṭūru[opopanax]n
1,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.004479,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
2,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000323,0.000014,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
3,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000296,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
4,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000119,0.000005,0.000005,...,0.000005,0.000005,0.000005,0.000094,0.000005,0.000005,0.000345,0.000549,0.000005,0.000005
5,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000410,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000159,0.000009,0.000009,0.000009,0.000009
6,0.000009,0.000009,0.000009,0.000009,0.000009,0.000070,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000299,0.000009
7,0.000008,0.000404,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
8,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000108,...,0.000014,0.000014,0.000014,0.000014,0.000111,0.000014,0.001660,0.000014,0.000014,0.000014
9,0.000498,0.000029,0.000498,0.000029,0.000498,0.000029,0.000029,0.000029,0.000029,0.000029,...,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029
10,0.000004,0.000004,0.000004,0.000028,0.000004,0.000004,0.000692,0.000329,0.000004,0.000004,...,0.000078,0.000004,0.000142,0.000004,0.000004,0.000004,0.005634,0.000004,0.000004,0.000061


In [19]:
#just checking
t_t_df['ēkallu[palace]n']

1     0.004932
2     0.006496
3     0.019528
4     0.000532
5     0.000031
6     0.004552
7     0.003679
8     0.003392
9     0.004021
10    0.009725
Name: ēkallu[palace]n, dtype: float64

# Visualize the Documents 1: Using MDS
While pyLDAvis is an excellent tool for exploring the topic/term aspect of a topic model (the words and their probabilities in each topic) it does not provide access to the document/topic aspect (the probability distribution of topics in each document). The visualization below plots all the documents according to their (cosine) distances (using Multi-Dimensional Scaling) in the Document/Term DataFrame. Each document (data point in the visualization) is colored according to the most prevalent topic and the size of the dot represents the probability of the most prevalent topic in that document.

Compute the distances between each of the documents. Use either the Document/Topic Dataframe or the Document/Term Dataframe (constructed below) to measure distance.

Since the data is already in list format, CountVectorizer does not need to preprocess or tokenize. The only way to prevent CountVectorizer from doing so is by creating dummy functions for the preprocessor and the tokenizer. These functions simply return the argument they receive.

In [20]:
cv = CountVectorizer(analyzer='word', preprocessor=lambda x: x, tokenizer=lambda x: x)
dtm = cv.fit_transform(texts)
dtm_df = pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names(), index = df.index.values)
dtm_df.head()

,aban-bāšti[(a-stone)]n,aban-lamassi[(a-precious-stone)]n,aban-râmi['love'-stone]n,abati[(meaning-unknown)]n,abašmû[(a-stone)]n,abbušu[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,ablu[brought]aj,...,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭīpu[addition]n,ṭīru[impression]n,ṭūbtu[peace]n,ṭūbu[goodness]n,ṭūbātu[happiness]n,ṭūdu[way]n,ṭūru[opopanax]n
P224378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P224382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P224383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P224386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P224388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
dist = squareform(pdist(dtm_df, 'cosine'))

Compute the position of each document using Multi-Dimensional Scaling. The variable `pos` holds the `x` and `y`  coordinates.

In [22]:
mds = MDS(n_components=2, max_iter=3000,
       random_state=seed, dissimilarity="precomputed", n_jobs=1)
pos = mds.fit_transform(dist)

Create lists of x and y values (coordinates).

In [23]:
mds_x = [x for x, y in pos]
mds_y = [y for x, y in pos]

Create lists of the most prevalent topic, the probability of the most prevalent topic, and the text name for each document. These lists are used in the tooltips of the Bokeh visualization.

In [24]:
prevalent_topic = d_t_df.idxmax(axis=1)
probability = d_t_df.max(axis=1)
designation = list(df['designation'])

In [25]:
len(prevalent_topic), len(probability), len(designation)

(3006, 3006, 3006)

# Define Colors

Create a colormap. 

In [26]:
colormap = {0: 'grey', 1: "orange", 2: "olive", 3: "firebrick", 
          4: "gold", 5: "red", 6: "fuchsia", 7: "green", 
          8: "blue", 9: "purple", 10: "aqua", 11: "yellow", 
          12: "indigo", 13: "blueviolet", 14: "beige", 15:"navy", 16: 'chocolate',
          17: 'azure', 18: 'coral', 19: 'crimson', 20: 'darkblue', 21: 'darkkhaki', 
          22: 'darkseagreen', 23: 'darkturquoise', 24: 'deeppink', 25: 'black'}

Create a dictionary, to be used by Bokeh for drawing the visualization. In the dictionary each key is a feature and each value is a list with the values of that feature for each data point. All lists (all values) should be of equal length (the number of data points). The features include x and y coordinates, color, size, etc. 

In [27]:
d_mds = dict(
        x=mds_x,
        y=mds_y,
        id_text=list(df.id_text),
        size = probability/max(probability)*15,
        probability = probability,
        topic = prevalent_topic,
        color = [colormap[n] for n in prevalent_topic],
        alpha = [0.5] * len(mds_x),
        designation = designation
    )
instructions = [
    "Highlight (color) one or two topics by moving the sliders. If both sliders are 0, all topics are colored.",
    "Hover over a data point for more information. Click on a data point to go to the document edition.",
    "Use the toolbar to zoom, pan, reset, or save as .png."]

# JavaScript Code
Interactive features in Bokeh, such as sliders, use a callback function that is activated when a certain event takes place. This event can be a mouse movement, a click, or a change in the slider. Custom callback functions need to be written in JavaScript.



In [28]:
code = """
        var data = source.data;
        topic = data['topic']
        for (i = 0; i < topic.length; i++) {
            data.alpha[i] = 0.5
            data.color[i] = cm[topic[i]]
            if ((topic1.value == 0) && (topic2.value == 0)) {
                continue;
            } else if ((topic[i] == topic1.value) || (topic[i] == topic2.value)) {
                continue;
            } else {
                data.color[i] = 'grey'
                data.alpha[i] = '0.1'
            } 
        }
        source.change.emit();
        """

Draw the visualization. The visualization provides various tools for further exploration:
- tooltips (provides topic, probability, text name and URL)
- box zoom
- wheel zoom
- pan
- reset
- link to document edition
- save the visualization

In addition, the visualization has two sliders that allow the user to select two topics.

In [29]:
def drawviz(data, title, outputfile):
    source_mds = ColumnDataSource(data=data)
    p = figure(
        plot_width=1000, plot_height=1000,
        tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
        title=title)
    p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("designation", "@designation")
        ]
        ))

    p.circle('x', 
         'y', 
         color='color', 
         fill_alpha='alpha', 
         size='size', 
         source=source_mds
         )
    p.axis.visible = False

    slider1 = Slider(start=0, end=ntopics, value=0, step=1, title="Topic A")
    slider2 = Slider(start=0, end=ntopics, value=0, step=1, title="Topic B")

    callback = CustomJS(args=dict(source=source_mds, topic1 = slider1, 
                              topic2 = slider2, cm = colormap), code = code)
    slider1.js_on_change('value', callback)
    slider2.js_on_change('value', callback)
    
    url = "http://oracc.museum.upenn.edu/@id_text"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)

    for line in instructions:
        p.add_layout(Title(text=line), 'below')

    layout = column(slider1, slider2, p)
    show(layout)
    output_file(outputfile)
    save(layout);

In [30]:
title = "Projection with MDS. Size of the circle represents prevalence of the topic."
outputfile = 'vis/mds1.html'
drawviz(d_mds, title, 'vis/mds1.html')

## Alternative: plotting based on Document/Topic table
The following visualization uses the same approach, but takes the document/topic table as the basis for distance measurements. Documents that share approximately the same distribution of topics will be plotted n the same region. Since the sum of each row in the document/topic table is 1 the distance matrix is computed with euclidean distance (not cosine).

In [31]:
dist_dt = squareform(pdist(d_t_df))

In [32]:
mds = MDS(n_components=2, max_iter=3000,
       random_state=seed, dissimilarity="precomputed", n_jobs=1)
pos = mds.fit_transform(dist_dt)

In [33]:
d_mds2 = d_mds # the data source is the same as for the previous visualization, except for the x and y coordinates.
d_mds2['x'] = [x for x, y in pos]
d_mds2['y'] = [y for x, y in pos]

In [34]:
title = "Projection with MDS, based on Document/Topic distribution. Size of the circle represents prevalence of the topic."
outputfile = 'vis/mds2.html'
drawviz(d_mds2, title, outputfile)

# Visualize the Documents 2: Using TSNE

# TSNE based on Document/Term Matrix (Cosine distance)

Cosine distances have been computed earlier; the matrix is stored in the variable `dist`.

In [35]:
X = dist
tsne = TSNE(n_components = 2, random_state=0, metric="precomputed")
X_tsne = tsne.fit_transform(X)

In [36]:
d_tsne = d_mds # the data source is the same as for the previous visualization, except for the x and y coordinates.
d_tsne['x'] = [x for x, y in X_tsne]
d_tsne['y'] = [y for x, y in X_tsne]

In [37]:
title = "Projection with tSNE. Size of the circle represents prevalence of the topic."
outputfile = 'vis/tsne1.html'
drawviz(d_tsne, title, outputfile)

# TSNE based on Document/Topic Matrix

In [38]:
X = dist_dt
tsne = TSNE(n_components = 2, random_state=0, metric="precomputed")
X_tsne = tsne.fit_transform(X)

In [39]:
d_tsne2 = d_mds # the data source is the same as for the previous visualization, except for the x and y coordinates.
d_tsne2['x'] = [x for x, y in X_tsne]
d_tsne2['y'] = [y for x, y in X_tsne]

In [40]:
title = "Projection with tSNE, based on Document/Topic distribution. Size of the circle represents prevalence of the topic."
outputfile = 'vis/tsne2.html'
drawviz(d_tsne2, title, outputfile)